In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from pickle import load
import re
import math
import numpy as np
import matplotlib.pyplot as plt
import wandb
from loguru import logger
from train_config import CFG
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=Warning)

CFG = CFG()

2023-11-17 23:42:07.165607: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-17 23:42:07.290772: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-17 23:42:07.820261: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-17 23:42:07.820290: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-17 23:42:07.823311: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [ ]:
try:  # detect TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')  # TPU detection
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:  # detect GPUs
    strategy = tf.distribute.get_strategy()  # default strategy that works on CPU and single GPU

REPLICAS = strategy.num_replicas_in_sync
print("Number of Accelerators: ", strategy.num_replicas_in_sync)

In [3]:
import pandas as pd

In [9]:
df = pd.read_csv(CFG.DATA / 'train.csv')

In [13]:
df

,kingdom,name,family,supercategory,phylum,order,genus,class,license,file_name,...,longitude,dset,dataset,file_path,specific_epithet,image_dir_name,image_id,class_id,gcs_path,class_priors
0,Fungi,Phallus indusiatus,Phallaceae,Fungi,Basidiomycota,Phallales,Phallus,Agaricomycetes,2,8d841f576d05e05f0b4b5513d549630a.jpg,...,-99.541750,train,2018,/home/broug/Desktop/Mushroom-Classifier/traini...,indusiatus,NaN,NaN,316,gs://mush-img-repo/data/raw/2018/Basidiomycota...,0.002409
1,Fungi,Phallus indusiatus,Phallaceae,Fungi,Basidiomycota,Phallales,Phallus,Agaricomycetes,3,f84f23fe93f3fde53f7193e3cc08d473.jpg,...,-95.171649,train,2018,/home/broug/Desktop/Mushroom-Classifier/traini...,indusiatus,NaN,NaN,316,gs://mush-img-repo/data/raw/2018/Basidiomycota...,0.002409
2,Fungi,Phallus indusiatus,Phallaceae,Fungi,Basidiomycota,Phallales,Phallus,Agaricomycetes,3,dabc337065c65a0ed19707c4a595bcb1.jpg,...,115.213867,train,2018,/home/broug/Desktop/Mushroom-Classifier/traini...,indusiatus,NaN,NaN,316,gs://mush-img-repo/data/raw/2018/Basidiomycota...,0.002409
3,Fungi,Phallus indusiatus,Phallaceae,Fungi,Basidiomycota,Phallales,Phallus,Agaricomycetes,3,8980532c8a1ef146bd3ed8d54f362b76.jpg,...,-93.019913,train,2018,/home/broug/Desktop/Mushroom-Classifier/traini...,indusiatus,NaN,NaN,316,gs://mush-img-repo/data/raw/2018/Basidiomycota...,0.002409
4,Fungi,Phallus indusiatus,Phallaceae,Fungi,Basidiomycota,Phallales,Phallus,Agaricomycetes,2,2d3e29631fa00733061111743ceb734c.jpg,...,-70.694447,train,2018,/home/broug/Desktop/Mushroom-Classifier/traini...,indusiatus,NaN,NaN,316,gs://mush-img-repo/data/raw/2018/Basidiomycota...,0.002409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101280,Fungi,Tremella mesenterica,Tremellaceae,Fungi,Basidiomycota,Tremellales,Tremella,Tremellomycetes,0,51f13b0b-f4f7-46ed-9e53-4d0c63ddad6b.jpg,...,35.868820,val,2021,/home/broug/Desktop/Mushroom-Classifier/traini...,mesenterica,05728_Fungi_Basidiomycota_Tremellomycetes_Trem...,2748639.0,435,gs://mush-img-repo/data/raw/2021/Basidiomycota...,0.003505
101281,Fungi,Tremella mesenterica,Tremellaceae,Fungi,Basidiomycota,Tremellales,Tremella,Tremellomycetes,1,294cf8dc-bfaa-45b7-a7a8-205d1a22e33d.jpg,...,143.641740,val,2021,/home/broug/Desktop/Mushroom-Classifier/traini...,mesenterica,05728_Fungi_Basidiomycota_Tremellomycetes_Trem...,2753098.0,435,gs://mush-img-repo/data/raw/2021/Basidiomycota...,0.003505
101282,Fungi,Tremella mesenterica,Tremellaceae,Fungi,Basidiomycota,Tremellales,Tremella,Tremellomycetes,4,b3b8cb90-d5d6-4118-826f-d69c6803a11a.jpg,...,-83.198750,val,2021,/home/broug/Desktop/Mushroom-Classifier/traini...,mesenterica,05728_Fungi_Basidiomycota_Tremellomycetes_Trem...,2754429.0,435,gs://mush-img-repo/data/raw/2021/Basidiomycota...,0.003505
101283,Fungi,Tremella mesenterica,Tremellaceae,Fungi,Basidiomycota,Tremellales,Tremella,Tremellomycetes,1,f38f1edd-34f2-4e8d-aaeb-c95ed22dd1ca.jpg,...,-78.370990,val,2021,/home/broug/Desktop/Mushroom-Classifier/traini...,mesenterica,05728_Fungi_Basidiomycota_Tremellomycetes_Trem...,2777349.0,435,gs://mush-img-repo/data/raw/2021/Basidiomycota...,0.003505


In [18]:
df['image_dir'] = df.apply(lambda x: f"{x['kingdom']}_{x['phylum']}_{x['class']}_{x['order']}_{x['family']}_{x['genus']}_{x['specific_epithet']}", axis=1)

In [22]:
df.groupby('image_dir').first()['class_id'].value_counts()

class_id
231    2
229    2
228    2
212    2
152    2
      ..
17     1
16     1
15     1
14     1
28     1
Name: count, Length: 467, dtype: int64

In [31]:
df['file_path'] = df.apply(lambda x: f"{CFG.DATA}/train/{x['image_dir']}/{x['file_name']}", axis=1)

In [32]:
df.to_csv(CFG.DATA / 'train.csv', index=False)